<a href="https://colab.research.google.com/github/cgrant00744/cgrant00744/blob/main/fantasy_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install espn.api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


In [10]:
import pandas as pd
from espn_api.football import League

# You'll need your ESPN cookies

league_id=128677568,
year=2024,
espn_s2='AEB4h%2F2UDDeSs6QS94SkWU7QE%2B3sRINvSf8B%2FswylrovuTYewJYMNEM1KaHWB%2FZzcRtRt42NanLouT%2B4ts86OZRb5QW%2Bf9%2FoibL0CFTGtcCmRjWSnhzg3V40XhiqlMZM5C0zeIiAIIhTUdVuP0taJc98MJU5DkfAbsGm%2B1eAaXO7%2FSCxV9fXLskO8FvLxpjRUPmVmDK7tXa6QA0KidTdeKrCiDqgmHnNq5meEmGBZm3mK9x9n6OtbRSMU5E0w0aCrYrZUFG7spw%2FReK1atnZxQZ9',
swid='{413A23DC-5421-451B-95E0-7348BE53FF56}'






In [14]:
def get_historical_data(league_id, years, espn_s2, swid):
    historical_data = {}

    for year in years:
        try:
            league = League(
                league_id=128677568,
                year=2024,
                espn_s2='AEB4h%2F2UDDeSs6QS94SkWU7QE%2B3sRINvSf8B%2FswylrovuTYewJYMNEM1KaHWB%2FZzcRtRt42NanLouT%2B4ts86OZRb5QW%2Bf9%2FoibL0CFTGtcCmRjWSnhzg3V40XhiqlMZM5C0zeIiAIIhTUdVuP0taJc98MJU5DkfAbsGm%2B1eAaXO7%2FSCxV9fXLskO8FvLxpjRUPmVmDK7tXa6QA0KidTdeKrCiDqgmHnNq5meEmGBZm3mK9x9n6OtbRSMU5E0w0aCrYrZUFG7spw%2FReK1atnZxQZ9',
                swid='413A23DC-5421-451B-95E0-7348BE53FF56'
            )
            historical_data[year] = league
            print(f"Successfully loaded {year} data")
        except Exception as e:
            print(f"Could not load {year}: {e}")

    return historical_data

# Get last 3 years of data
years = [2024]
all_leagues = get_historical_data(league_id, years, espn_s2, swid)

Successfully loaded 2024 data


In [17]:
def detailed_leagues_to_dataframe(all_leagues):
    all_data = []

    for year, league in all_leagues.items():
        for team in league.teams:
            for player in team.roster:
                # Get additional stats if available
                player_data = {
                    'year': year,
                    'player_name': player.name,
                    'team_name': team.team_name,
                    'position': getattr(player, 'position', 'Unknown'),
                    'total_points': player.total_points,
                    'projected_points': getattr(player, 'projected_total_points', None),
                    'avg_points': getattr(player, 'avg_points', None),
                    'player_id': player.playerId,
                    'pro_team': getattr(player, 'proTeam', 'Unknown'),
                    'injured': getattr(player, 'injured', False),
                    'injury_status': getattr(player, 'injuryStatus', 'Healthy')
                }
                all_data.append(player_data)

    return pd.DataFrame(all_data)

df_detailed = detailed_leagues_to_dataframe(all_leagues)

In [21]:
df_detailed.head()

,year,player_name,team_name,position,total_points,projected_points,avg_points,player_id,pro_team,injured,injury_status
0,2024,Jahmyr Gibbs,Sending Dak picks to my Nabers,RB,369.9,241.30,21.76,4429795,DET,False,ACTIVE
1,2024,Michael Pittman Jr.,Sending Dak picks to my Nabers,WR,167.8,234.46,10.49,4035687,IND,False,QUESTIONABLE
2,2024,DJ Moore,Sending Dak picks to my Nabers,WR,240.1,226.93,14.12,3915416,CHI,False,ACTIVE
3,2024,Nico Collins,Sending Dak picks to my Nabers,WR,215.6,233.23,17.97,4258173,HOU,False,ACTIVE
4,2024,Aaron Jones,Sending Dak picks to my Nabers,RB,243.6,192.10,14.33,3042519,MIN,False,ACTIVE


In [22]:
df_detailed.to_excel('fantasy_football_data.xlsx', index=False)

In [23]:
# Download the file
from google.colab import files
files.download('fantasy_football_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
def get_2025_projections_df(league):
    projection_data = []

    # Get rostered players
    for team in league.teams:
        for player in team.roster:
            if hasattr(player, 'projected_total_points'):
                projection_data.append({
                    'player_name': player.name,
                    'position': getattr(player, 'position', 'Unknown'),
                    'pro_team': getattr(player, 'proTeam', 'Unknown'),
                    'projected_points': player.projected_total_points,
                    'status': 'Rostered',
                    'fantasy_team': team.team_name
                })


    return pd.DataFrame(projection_data)

# Create the DataFrame
projections_df = get_2025_projections_df(league)


In [29]:
projections_df.head()

,player_name,position,pro_team,projected_points,status,fantasy_team
0,Jahmyr Gibbs,RB,DET,241.30,Rostered,Sending Dak picks to my Nabers
1,Michael Pittman Jr.,WR,IND,234.46,Rostered,Sending Dak picks to my Nabers
2,DJ Moore,WR,CHI,226.93,Rostered,Sending Dak picks to my Nabers
3,Nico Collins,WR,HOU,233.23,Rostered,Sending Dak picks to my Nabers
4,Aaron Jones,RB,MIN,192.10,Rostered,Sending Dak picks to my Nabers


In [30]:
  projections_df.to_excel('projections.xlsx', index=False)

In [32]:
files.download('projections.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
import requests
from bs4 import BeautifulSoup


def scrape_fantasypros_projections(position='qb',year=2025):
    """
    Scrape FantasyPros consensus projections
    Positions: qb, rb, wr, te, k, dst
    """
    url = f"https://www.fantasypros.com/nfl/projections/{position}.php"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the projections table
        table = soup.find('table', {'id': 'data'})

        if table:
            # Convert to DataFrame
            df = pd.read_html(str(table))[0]
            df['position'] = position.upper()
            return df
        else:
            print(f"Could not find table for {position}")
            return pd.DataFrame()

    except Exception as e:
        print(f"Error scraping {position}: {e}")
        return pd.DataFrame()

# Get projections for all positions
positions = ['qb', 'rb', 'wr', 'te']
all_positions_df = []

for pos in positions:
    pos_df = scrape_fantasypros_projections(pos)
    if not pos_df.empty:
        all_positions_df.append(pos_df)

    # Be respectful - add delay between requests
    import time
    time.sleep(1)

# Combine all positions
if all_positions_df:
    combined_projections = pd.concat(all_positions_df, ignore_index=True)
    print(combined_projections.head())

/tmp/ipython-input-2766787128.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/tmp/ipython-input-2766787128.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/tmp/ipython-input-2766787128.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/tmp/ipython-input-2766787128.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


   Unnamed: 0_level_0 PASSING                        RUSHING            MISC  \
               Player     ATT   CMP    YDS  TDS INTS     ATT   YDS  TDS   FL   
0     Jalen Hurts PHI    27.8  18.6  211.7  1.4  0.6     9.1  37.6  0.9  0.2   
1      Josh Allen BUF    32.8  20.8  238.6  1.6  0.7     6.5  31.9  0.6  0.2   
2   Lamar Jackson BAL    29.4  19.6  226.4  1.8  0.6     8.3  49.4  0.2  0.3   
3  Jayden Daniels WAS    31.4  20.7  228.5  1.5  0.7     8.8  46.6  0.4  0.2   
4      Joe Burrow CIN    35.2  24.5  269.7  2.1  0.7     3.0  11.1  0.1  0.2   

        position RECEIVING          
   FPTS                REC YDS TDS  
0  22.0       QB       NaN NaN NaN  
1  21.8       QB       NaN NaN NaN  
2  21.6       QB       NaN NaN NaN  
3  21.2       QB       NaN NaN NaN  
4  20.0       QB       NaN NaN NaN  


In [43]:
combined_projections.to_excel('projections.xlsx')

In [44]:
files.download('projections.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>